In [ ]:
import matplotlib.pyplot as plt
import numpy as np

## 3+D surface plotting

Many many examples online:
* https://matplotlib.org/stable/gallery/mplot3d/index.html

Just one example: 3D quiver plot

In [ ]:
ax = plt.figure().add_subplot(projection="3d")

# Make the grid
x, y, z = np.meshgrid(
    np.arange(-0.8, 1, 0.3), np.arange(-0.8, 1, 0.3), np.arange(-0.8, 1, 0.3)
)

# Make the direction data for the arrows
u = np.sin(np.pi * x) * np.cos(np.pi * y) * np.cos(np.pi * z)
v = -np.cos(np.pi * x) * np.sin(np.pi * y) * np.cos(np.pi * z)
w = np.sqrt(2.0 / 3.0) * np.cos(np.pi * x) * np.cos(np.pi * y) * np.sin(np.pi * z)

ax.quiver(x, y, z, u, v, w, length=0.1, normalize=True)

plt.show()

---------------

## Galaxy data

The `galaxy.csv` data file contains measurements from a galaxy. We are interested in the velocity (calculated from the observed redshift) as a function of angular

* **angles** -- slit angles of telescope (7 unique angles)
* **arcsecEW** --  east-west coordinate of observations (323 bearings)
* **arcsecNS** -- north-south coordinate of observations (323 bearings)
* **radialPos** -- radial position along the slit angle
* **velocity** -- velocity of galaxy (323 readings)

Data from:

_The structure and dynamics of ringed galaxies. III - Surface photometry and kinematics of the ringed nonbarred spiral NGC 7531_, R. Buta, Astrophysical Journal Supplement Series, **64**, May 1987, p. 1-37.

* https://adsabs.harvard.edu/full/1987ApJS...64....1B

In [ ]:
data = np.genfromtxt("galaxy.csv", delimiter=",", names=True, skip_header=1)
cols = list(data.dtype.names)
print(cols)

In [ ]:
%matplotlib inline
plt.scatter(data["arcsecEW"], data["arcsecNS"], marker="x")
plt.show()

In [ ]:
%matplotlib inline

plt.scatter(
    data["arcsecEW"], data["arcsecNS"], marker="x", c=data["velocity"], cmap="coolwarm"
)
plt.colorbar(label="velocity")
plt.xlabel("EW")
plt.ylabel("NS")
plt.show()

In [ ]:
# %matplotlib tk
%matplotlib inline

ax = plt.axes(projection="3d")
ax.view_init(60.0, -130.0, 0)
cb = ax.scatter(
    data["arcsecEW"],
    data["arcsecNS"],
    data["velocity"],
    marker="x",
    c=data["velocity"],
    cmap="coolwarm",
)
ax.set(xlabel="EW", ylabel="NS")
plt.colorbar(cb, label="velocity")
plt.show()

### Surface: plot_trisurf

In [ ]:
# %matplotlib tk
%matplotlib inline

ax = plt.axes(projection="3d")
ax.view_init(60.0, -130.0, 0)
ax.scatter(
    data["arcsecEW"],
    data["arcsecNS"],
    data["velocity"],
    marker="x",
    c=data["velocity"],
    cmap="coolwarm",
)
ax.plot_trisurf(
    data["arcsecEW"], data["arcsecNS"], data["velocity"], color="grey", alpha=0.3
)
ax.set(xlabel="EW", ylabel="NS", zlabel="velocity")
plt.show()

### Wire mesh (using interpolate/griddata)

* https://docs.scipy.org/doc/scipy/reference/generated/scipy.interpolate.griddata.html

method: nearest, linear, cubic

Always be very careful interpolating: can be unwanted artifacts

A good rule of thing: always at least check the plot with data and interpolated function together!

In [ ]:
%matplotlib inline

from scipy.interpolate import griddata
import matplotlib.pyplot as plt

x0, xmax = min(data["arcsecEW"]), max(data["arcsecEW"])
y0, ymax = min(data["arcsecNS"]), max(data["arcsecNS"])

# Create fine grid of points
xs = np.linspace(x0, xmax, 100)
ys = np.linspace(y0, ymax, 100)
X, Y = np.meshgrid(xs, ys)

# interpolate data onto our fine grid
Z = griddata(
    np.column_stack((data["arcsecEW"], data["arcsecNS"])),
    data["velocity"],
    (X, Y),
    method="linear",
)


ax = plt.axes(projection="3d")

ax.view_init(60.0, -130.0, 0)
ax.plot_wireframe(X, Y, Z, rstride=2, cstride=8, color="grey", alpha=0.5)
ax.scatter(data["arcsecEW"], data["arcsecNS"], data["velocity"], color="b", marker="x")
ax.set(xlabel="EW", ylabel="NS", zlabel="velocity")
plt.show()

In [ ]:
# %matplotlib tk
%matplotlib inline

ax = plt.axes(projection="3d")

ax.view_init(60.0, -130.0, 0)
ax.plot_surface(
    X,
    Y,
    Z,
    cmap="coolwarm",
    edgecolor="royalblue",
    lw=0.5,
    rstride=5,
    cstride=5,
    alpha=1,
)
ax.scatter(
    data["arcsecEW"],
    data["arcsecNS"],
    data["velocity"],
    color="b",
    marker="x",
    alpha=0.3,
)
ax.set(xlabel="EW", ylabel="NS", zlabel="velocity")
plt.show()

### Contours (using interpolated data)

In [ ]:
%matplotlib inline
plt.pcolormesh(X, Y, Z, cmap="coolwarm", shading="gouraud")
plt.colorbar(label="velocity")
cs = plt.contour(X, Y, Z, 7, colors="k", linewidths=2)
plt.clabel(cs, inline=True)
plt.scatter(data["arcsecEW"], data["arcsecNS"], marker="x", c="k", alpha=0.2)
plt.xlabel("EW")
plt.ylabel("NS")
plt.show()

### Contours in 3D

In [ ]:
# %matplotlib tk
%matplotlib inline

ax = plt.axes(projection="3d")

ax.view_init(50.0, -10.0)


ax.contour(X, Y, Z, extend3d=False, cmap="coolwarm")

ax.set(xlabel="EW", ylabel="NS", zlabel="velocity")
plt.show()

In [ ]:
# %matplotlib tk
%matplotlib inline

ax = plt.axes(projection="3d")

ax.view_init(50.0, -10.0)

cs = ax.contourf(X, Y, Z, cmap="coolwarm")

plt.colorbar(cs, orientation="horizontal", fraction=0.05)
ax.set(xlabel="EW", ylabel="NS", zlabel="velocity")
plt.show()

### Combine: Project contour plot onto lower axis

In [ ]:
# %matplotlib tk
%matplotlib inline

ax = plt.axes(projection="3d")

ax.view_init(40.0, -30.0)
ax.set_xlim(-40, 40)
ax.set_ylim(-50, 50)
ax.set_zlim(1300, 1800)
ax.plot_surface(
    X,
    Y,
    Z,
    color="lightblue",
    edgecolor="royalblue",
    lw=0.5,
    rstride=5,
    cstride=5,
    alpha=0.5,
)
ax.scatter(
    data["arcsecEW"],
    data["arcsecNS"],
    data["velocity"],
    color="black",
    marker="x",
    alpha=0.5,
)

# zdir makes projection of contour plot
# offset puts it where we want it (on one of the axis)
cs = ax.contourf(X, Y, Z, zdir="z", offset=1300, cmap="coolwarm")

plt.colorbar(cs, orientation="horizontal", fraction=0.05)
ax.set(xlabel="EW", ylabel="NS", zlabel="velocity")
plt.show()